# Test skill

In [ ]:
from characters.ken import Ken
import time

# Choose attack mode: 'modern' or 'classic'
ken = Ken(attack_mode='modern')

def test_basic_attack():
    ken.impl.light_attack()     # Perform light attack
    time.sleep(1)
    ken.impl.medium_attack()    # Perform medium attack
    time.sleep(1)
    ken.impl.heavy_attack()     # Perform heavy attack
    time.sleep(1)
    ken.impl.special_attack()   # Perform heavy attack
    time.sleep(1)
    
def test_other_commands():
    ken.impl.drive_parry()      # defense
    time.sleep(1)
    ken.impl.drive_impact()     # Drive Impact
    time.sleep(2)
    ken.impl.throw()            # Throw
    time.sleep(1)
    ken.impl.assist()           # OC
    time.sleep(1)

def test_move_left_right_crouch():
    ken.impl.move_jump()
    time.sleep(3)
    
    # Move Left
    ken.impl.move_left_continuously()   
    time.sleep(3)
    ken.impl.stop_movement()
    time.sleep(3)
    
    # Move Right
    ken.impl.move_right_continuously()
    time.sleep(3)
    ken.impl.stop_movement()
    time.sleep(3)
    
    # crouch
    ken.impl.move_crouch_continuously()
    time.sleep(3)
    ken.impl.stop_movement()
    time.sleep(3)

def test_up_down_left_right():
    # down left
    ken.impl.down_left_continuously()
    time.sleep(2)
    ken.impl.stop_movement()
    time.sleep(2)
    
    # down right
    ken.impl.down_right_continuously()
    time.sleep(2)
    ken.impl.stop_movement()
    time.sleep(2)
    
    # up left
    ken.impl.up_left_continuously()
    time.sleep(2)
    ken.impl.stop_movement()
    
    # up right
    ken.impl.up_right_continuously()
    time.sleep(2)
    ken.impl.stop_movement()

def test_ken_special_skills():
    print("Testing Ken's basic special moves...")

if __name__ == "__main__":
    time.sleep(2)
    ken = Ken(attack_mode='modern')
    for n in range(3):
        ken.thunder_kick()
        time.sleep(1)
        

# Detector Player HP

In [ ]:
import cv2
import keyboard
from utils.screen_capture import screen_capture
from utils.player_hp_detector import L_player_HP, R_player_HP

while True:
    # Capture game screen
    screen = screen_capture((0, 0, 1280, 720))
    
    # player_HP = screen_capture((115, 85, 1165, 100))
    player_HP = screen[85:100, 115:1165]
    
    cv2.imshow("Player HP", player_HP)
    
    L_player_HP_part = player_HP[:, 15:(int(player_HP.shape[1] / 2) - 77)]
    cv2.imshow("Left Player HP", L_player_HP_part)
    
    R_player_HP_part = player_HP[:, (int(player_HP.shape[1] / 2) + 77):-15]
    cv2.imshow("Right Player HP", R_player_HP_part)
    
    print(f"\rLeft player HP: {L_player_HP(player_HP)}, Right player HP: {R_player_HP(player_HP)}", end='')
    
    
    if cv2.waitKey(5) & keyboard.is_pressed('q'):
        break
cv2.destroyAllWindows()

# Detector Player Drive Gauge

In [ ]:
import cv2
import keyboard
import os
import time
from utils.screen_capture import screen_capture

time.sleep(3)

# Global variable to keep track of the index
file_name = "0"
def save_image(img1, img2, index):
    # Define the save paths for the images with the current index
    save_path_L = f"game_state_recognition/data/drive_gauge/{file_name}/L_{index}.png"
    save_path_R = f"game_state_recognition/data/drive_gauge/{file_name}/R_{index}.png"
    
    # Save the images
    cv2.imwrite(save_path_L, img1)
    cv2.imwrite(save_path_R, img2)
    print(f"\rImages saved: {save_path_L}, {save_path_R}", end='')

while True:
    # Capture the screen portion for drive gauge
    player_DriveGauge = screen_capture((375, 115, 910, 125))
    
    # Crop the left and right sections of the drive gauge
    L_player_DriveGauge = player_DriveGauge[:, :223]
    cv2.imshow("L Player Power", L_player_DriveGauge)
    
    R_player_DriveGauge = player_DriveGauge[:, 310:]
    cv2.imshow("R Player Power", R_player_DriveGauge)

    # save_image(L_player_DriveGauge, R_player_DriveGauge, index)

    # Wait for 1 ms and break if 'q' is pressed
    cv2.waitKey(1)
    if keyboard.is_pressed('q'):
        break

cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
import cv2
import keyboard
import os
import time
from utils.screen_capture import screen_capture

# 載入訓練好的分類模型
model = YOLO('drive_gauge_project/yolov8_drive_gauge5/weights/best.pt')

while True:
    # 擷取整個斗氣槽區域
    player_DriveGauge = screen_capture((375, 115, 910, 125))
    
    # 分割左右玩家的斗氣槽圖像
    L_player_DriveGauge = player_DriveGauge[:, :223]
    R_player_DriveGauge = player_DriveGauge[:, 310:]

    # 推論（模型為分類任務）
    L_result = model.predict(L_player_DriveGauge, imgsz=64, device=0, verbose=False)
    R_result = model.predict(R_player_DriveGauge, imgsz=64, device=0, verbose=False)

    # 取得分類結果（分類任務只會有一個結果）
    L_class = L_result[0].probs.top1
    L_conf = L_result[0].probs.data[L_class].item()
    
    R_class = R_result[0].probs.top1
    R_conf = R_result[0].probs.data[R_class].item()

    # 打印預測結果
    print(f"\rL Player Drive Gauge: Class {L_class} (Confidence: {L_conf:.2f}), R Player Drive Gauge: Class {R_class} (Confidence: {R_conf:.2f})", end='')

    # 顯示圖像
    cv2.imshow("L Player Power", L_player_DriveGauge)
    cv2.imshow("R Player Power", R_player_DriveGauge)

    cv2.waitKey(1)
    if keyboard.is_pressed('q'):
        break

cv2.destroyAllWindows()


# Detector Player player SuperArtCA

In [ ]:
import cv2
import keyboard
import time
from utils.screen_capture import screen_capture

time.sleep(3)

# Global variable to keep track of the index
index = 0
file_name = "CA"
def save_image(img1, img2, index):
    # Define the save paths for the images with the current index
    save_path_L = f"game_state_recognition/data/super_art/{file_name}/L_{index}.png"
    save_path_R = f"game_state_recognition/data/super_art/{file_name}/R_{index}.png"
    
    # Save the images
    cv2.imwrite(save_path_L, img1)
    cv2.imwrite(save_path_R, img2)
    print(f"Images saved: {save_path_L}, {save_path_R}")
    
for n in range(1001):
# while True:
    player_DriveGauge = screen_capture((50, 675, 1230, 730))
    L_player_DriveGauge = player_DriveGauge[:, :60]
    R_player_DriveGauge = player_DriveGauge[:, 1125:]

    cv2.imshow("Left Gauge", L_player_DriveGauge)
    cv2.imshow("Right Gauge", R_player_DriveGauge)
    
    save_image(L_player_DriveGauge, R_player_DriveGauge, index)
    
    index += 1
    
    # Break loop if 'q' is pressed
    cv2.waitKey(1)
    if keyboard.is_pressed('q'):
        print("退出程式...")
        break

cv2.destroyAllWindows()

# Mouse Color Tracker

In [ ]:
import cv2
from utils.screen_color_tracker import MouseColorTracker

if __name__ == "__main__":
    tracker = MouseColorTracker(region_size=10)
    
    print("Press q to stop tracking.")
    try:
        while True:
            # tracker.get_color_rgb()  # Uncomment to track RGB colors
            tracker.get_color_hsv()  # Track HSV colors
            cv2.waitKey(100)
            if keyboard.is_pressed('q'):
                break
    except KeyboardInterrupt:
        print("\nTracking ended.")